### BELOW CODE IS TO WORK WITH SPARK IN COLAB, IGNORE IF NOT USING COLAB

In [2]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,174 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,081 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,374 kB]
Get:13 http://archive.ubuntu.com/ubuntu jam

In [3]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON']=sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']=sys.executable

In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('week4').getOrCreate()
spark

### 1) Demonstrate how to load a dataset suitable for recommendation systems into a PySpark DataFrame.





In [5]:
path='/content/movies 1.json'
data=spark.read.json(path)
data.show()

+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+
|helpfulness|product_id|        profile_name|              review|score|             summary|      time|       user_id|
+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+
|        7/7|B003AI2VGA|Brian E. Erland "...|Synopsis: On the ...|  3.0|"There Is So Much...|1182729600|A141HP4LYPWMSR|
|        4/4|B003AI2VGA|          Grady Harp|THE VIRGIN OF JUA...|  3.0|Worthwhile and Im...|1181952000|A328S9RN3U5M68|
|       8/10|B003AI2VGA|Chrissy K. McVay ...|The scenes in thi...|  5.0|This movie needed...|1164844800|A1I7QGUDP043DG|
|        1/1|B003AI2VGA|        golgotha.gov|THE VIRGIN OF JUA...|  3.0|distantly based o...|1197158400|A1M5405JH9THP9|
|        1/1|B003AI2VGA|KerrLines "&#34;M...|Informationally, ...|  3.0|"What's going on ...|1188345600| ATXL536YX71TR|
|        0/0|B003AI2VGA|abra "a devoted 

out of all the columns in dataset, only userid, movieid and score contribute to giving information about what the users like. so we discaard everything and consider only those.

In [17]:
df=data.select("user_id","product_id","score")
df.show(5)

+--------------+----------+-----+
|       user_id|product_id|score|
+--------------+----------+-----+
|A141HP4LYPWMSR|B003AI2VGA|  3.0|
|A328S9RN3U5M68|B003AI2VGA|  3.0|
|A1I7QGUDP043DG|B003AI2VGA|  5.0|
|A1M5405JH9THP9|B003AI2VGA|  3.0|
| ATXL536YX71TR|B003AI2VGA|  3.0|
+--------------+----------+-----+
only showing top 5 rows



to fit to model,we need numerical values. so we convert categorical values of user and product IDs to numerical values using indexer.

In [20]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
user_indexer=StringIndexer(inputCol='user_id',outputCol='user_id_indexed')
product_indexer=StringIndexer(inputCol='product_id',outputCol='prod_id_indexed')
indexing=Pipeline(stages=[user_indexer,product_indexer])
df_indexed=indexing.fit(df).transform(df)
df_indexed.show(5)

+--------------+----------+-----+---------------+---------------+
|       user_id|product_id|score|user_id_indexed|prod_id_indexed|
+--------------+----------+-----+---------------+---------------+
|A141HP4LYPWMSR|B003AI2VGA|  3.0|           32.0|          731.0|
|A328S9RN3U5M68|B003AI2VGA|  3.0|            3.0|          731.0|
|A1I7QGUDP043DG|B003AI2VGA|  5.0|          312.0|          731.0|
|A1M5405JH9THP9|B003AI2VGA|  3.0|        10917.0|          731.0|
| ATXL536YX71TR|B003AI2VGA|  3.0|          173.0|          731.0|
+--------------+----------+-----+---------------+---------------+
only showing top 5 rows



### 2) Implement a PySpark script that splits the data and trains a recommendation model. and 3) Implement a PySpark script using the ALS algorithm for collaborative filtering.

In [22]:
from pyspark.ml.recommendation import ALS
train,test=df_indexed.randomSplit([0.8,0.2])
als=ALS(
    userCol='user_id_indexed',
    itemCol='prod_id_indexed',
    ratingCol='score',
    rank=10,
    regParam=0.01,
    coldStartStrategy='drop'
)
model=als.fit(train)

In [23]:
predictions=model.transform(test)
predictions.show(5)

+--------------+----------+-----+---------------+---------------+----------+
|       user_id|product_id|score|user_id_indexed|prod_id_indexed|prediction|
+--------------+----------+-----+---------------+---------------+----------+
| AF61MRUO1WLFN|B003U6SJXQ|  2.0|         5338.0|          148.0|0.45314217|
|  AIMR915K4YCN|B003U6SJXQ|  4.0|           15.0|          148.0|  2.509585|
|A27H9DOUGY9FOS|B003U6SJXQ|  4.0|           59.0|          148.0|  3.929099|
|A3M6RVUVZOQZLB|B003U6SJXQ|  5.0|         1000.0|          148.0|  3.291792|
|A1UOTG037037XY|B003U6SJXQ|  2.0|          181.0|          148.0|  8.228476|
+--------------+----------+-----+---------------+---------------+----------+
only showing top 5 rows



### 4) Implement code to evaluate the performance of the recommendation model using appropriate metrics.

In [24]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(predictionCol="prediction",labelCol="score",metricName='rmse')
loss=evaluator.evaluate(predictions)
print(f"loss={loss}")

loss=3.887200062195211


users and movies recommended for them

In [25]:
userRecs = model.recommendForAllUsers(5)
userRecs.show()

+---------------+--------------------+
|user_id_indexed|     recommendations|
+---------------+--------------------+
|            148|[{541, 21.881218}...|
|            463|[{491, 17.087206}...|
|            471|[{577, 33.52524},...|
|            496|[{770, 21.761837}...|
|            833|[{587, 20.524448}...|
|           1088|[{774, 8.919004},...|
|           1238|[{554, 6.2746067}...|
|           1342|[{394, 14.19214},...|
|           1580|[{540, 16.189127}...|
|           1591|[{661, 14.073083}...|
|           1645|[{700, 2.716014},...|
|           1829|[{547, 5.6317167}...|
|           1959|[{506, 10.049072}...|
|           2122|[{665, 6.8981977}...|
|           2142|[{255, 4.9989877}...|
|           2366|[{427, 12.485628}...|
|           2659|[{733, 10.706466}...|
|           2866|[{656, 15.0777445...|
|           3175|[{770, 14.365597}...|
|           3749|[{427, 10.162886}...|
+---------------+--------------------+
only showing top 20 rows



movies and users that might like the movies

In [26]:
itemRecs = model.recommendForAllItems(5)
itemRecs.show()

+---------------+--------------------+
|prod_id_indexed|     recommendations|
+---------------+--------------------+
|            471|[{102, 21.59742},...|
|           1342|[{152, 3.9051003}...|
|            463|[{67, 24.56299}, ...|
|            833|[{119, 29.114319}...|
|            496|[{270, 27.600008}...|
|            148|[{498, 12.221058}...|
|           1088|[{86, 5.74053}, {...|
|           1238|[{328, 13.444728}...|
|            540|[{164, 45.41156},...|
|           1460|[{350, 7.7674236}...|
|            392|[{77, 35.093914},...|
|           1522|[{106, 4.5277824}...|
|            243|[{164, 19.136005}...|
|            623|[{86, 28.25417}, ...|
|           1483|[{156, 19.715994}...|
|           1084|[{99, 12.665609},...|
|           1025|[{86, 37.33557}, ...|
|           1395|[{124, 11.546415}...|
|            737|[{454, 22.694351}...|
|            897|[{46, 19.376152},...|
+---------------+--------------------+
only showing top 20 rows

